In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200
%matplotlib inline

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vhuang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
wpt = nltk.WordPunctTokenizer()

In [8]:
stop_words = nltk.corpus.stopwords.words('english')

In [14]:
stop_words[10:20]

["you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [37]:
def normalize_document(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [ ]:
#below is testing example

In [20]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]

In [21]:
corpus = np.array(corpus)

In [22]:
corpus

array(['The sky is blue and beautiful.',
       'Love this blue and beautiful sky!',
       'The quick brown fox jumps over the lazy dog.',
       "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
       'I love green eggs, ham, sausages and bacon!',
       'The brown fox is quick and the blue dog is lazy!',
       'The sky is very blue and the sky is very beautiful today',
       'The dog is lazy but the brown fox is quick!'], dtype='<U66')

In [38]:
corpus[4]

'I love green eggs, ham, sausages and bacon!'

In [39]:
doc = re.sub(r'[^a-zA-Z\s]', '', corpus[4], re.I|re.A)
doc

'I love green eggs ham sausages and bacon'

In [40]:
doc = doc.lower()
doc

'i love green eggs ham sausages and bacon'

In [41]:
doc = doc.strip()
doc

'i love green eggs ham sausages and bacon'

In [56]:
tokens = wpt.tokenize(doc)
tokens

['i', 'love', 'green', 'eggs', 'ham', 'sausages', 'and', 'bacon']

In [57]:
filtered_tokens = [token for token in tokens if token not in stop_words]
filtered_tokens

['love', 'green', 'eggs', 'ham', 'sausages', 'bacon']

In [61]:
doc = ' '.join(filtered_tokens)
doc

'love green eggs ham sausages bacon'

In [62]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U51')

# Word Embeddings

In [63]:
from nltk.corpus import gutenberg
from string import punctuation

In [65]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [66]:
gutenberg

<PlaintextCorpusReader in '.../corpora/gutenberg' (not loaded yet)>